# Basic Text Processing with Spacy
    
You're a consultant for [DelFalco's Italian Restaurant](https://defalcosdeli.com/index.html).
The owner asked you to identify whether there are any foods on their menu that diners find disappointing. 

<img src="https://i.imgur.com/8DZunAQ.jpg" alt="Meatball Sub" width="250"/>

Before getting started, run the following cell to set up code checking.

In [1]:
import pandas as pd

Setup Complete


The business owner suggested you use diner reviews from the Yelp website to determine which dishes people liked and disliked. You pulled the data from Yelp. Before you get to analysis, run the code cell below for a quick look at the data you have to work with.

In [2]:
# Load in the data from JSON file
data = pd.read_json('../input/nlp-course/restaurant.json')
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
109,lDJIaF4eYRF4F7g6Zb9euw,lb0QUR5bc4O-Am4hNq9ZGg,r5PLDU-4mSbde5XekTXSCA,4,2,0,0,I used to work food service and my manager at ...,2013-01-27 17:54:54
1013,vvIzf3pr8lTqE_AOsxmgaA,MAmijW4ooUzujkufYYLMeQ,r5PLDU-4mSbde5XekTXSCA,4,0,0,0,We have been trying Eggplant sandwiches all ov...,2015-04-15 04:50:56
1204,UF-JqzMczZ8vvp_4tPK3bQ,slfi6gf_qEYTXy90Sw93sg,r5PLDU-4mSbde5XekTXSCA,5,1,0,0,Amazing Steak and Cheese... Better than any Ph...,2011-03-20 00:57:45
1251,geUJGrKhXynxDC2uvERsLw,N_-UepOzAsuDQwOUtfRFGw,r5PLDU-4mSbde5XekTXSCA,1,0,0,0,Although I have been going to DeFalco's for ye...,2018-07-17 01:48:23
1354,aPctXPeZW3kDq36TRm-CqA,139hD7gkZVzSvSzDPwhNNw,r5PLDU-4mSbde5XekTXSCA,2,0,0,0,"Highs: Ambience, value, pizza and deserts. Thi...",2018-01-21 10:52:58


The owner also gave you this list of menu items and common alternate spellings.

In [3]:
menu = ["Cheese Steak", "Cheesesteak", "Steak and Cheese", "Italian Combo", "Tiramisu", "Cannoli",
        "Chicken Salad", "Chicken Spinach Salad", "Meatball", "Pizza", "Pizzas", "Spaghetti",
        "Bruchetta", "Eggplant", "Italian Beef", "Purista", "Pasta", "Calzones",  "Calzone",
        "Italian Sausage", "Chicken Cutlet", "Chicken Parm", "Chicken Parmesan", "Gnocchi",
        "Chicken Pesto", "Turkey Sandwich", "Turkey Breast", "Ziti", "Portobello", "Reuben",
        "Mozzarella Caprese",  "Corned Beef", "Garlic Bread", "Pastrami", "Roast Beef",
        "Tuna Salad", "Lasagna", "Artichoke Salad", "Fettuccini Alfredo", "Chicken Parmigiana",
        "Grilled Veggie", "Grilled Veggies", "Grilled Vegetable", "Mac and Cheese", "Macaroni",  
         "Prosciutto", "Salami"]

# Step 1: Plan Your Analysis

Given the data from Yelp and the list of menu items, do you have any ideas for how you could find which menu items have disappointed diners?

Think about your answer. Then run the cell below to see one approach.

In [4]:
# Check your answer (Run this code cell to receive credit!)
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> You could group reviews by what menu items they mention, and then calculate the average rating
    for reviews that mentioned each item. You can tell which foods are mentioned in reviews with low scores,
    so the restaurant can fix the recipe or remove those foods from the menu.

# Step 2: Find items in one review

You'll pursue this plan of calculating average scores of the reviews mentioning each menu item.

As a first step, you'll write code to extract the foods mentioned in a single review.

Since menu items are multiple tokens long, you'll use `PhraseMatcher` which can match series of tokens.

Fill in the `____` values below to get a list of items matching a single menu item.

In [5]:
import spacy
from spacy.matcher import PhraseMatcher

index_of_review_to_test_on = 14
text_to_test_on = data.text.iloc[index_of_review_to_test_on]

# Load the SpaCy model
nlp = spacy.blank('en')

# Create the tokenized version of text_to_test_on
review_doc = nlp(text_to_test_on)

# Create the PhraseMatcher object. The tokenizer is the first argument. Use attr = 'LOWER' to make consistent capitalization
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

# Create a list of tokens for each item in the menu
menu_tokens_list = [nlp(item) for item in menu]

# Add the item patterns to the matcher. 
# Look at https://spacy.io/api/phrasematcher#add in the docs for help with this step
# Then uncomment the lines below 

matcher.add("MENU", None, *menu_tokens_list)
matches = matcher(review_doc)

print(matches)
# Uncomment to check your work
q_2.check()

[(8291075388056826051, 2, 3), (8291075388056826051, 16, 17), (8291075388056826051, 58, 59)]


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
# q_2.solution()

After implementing the above cell, uncomment the following cell to print the matches.

In [6]:
for match in matches:
   print(f"Token number {match[1]}: {review_doc[match[1]:match[2]]}")

Token number 2: Purista
Token number 16: prosciutto
Token number 58: meatball


# Step 3: Matching on the whole dataset

Now run this matcher over the whole dataset and collect ratings for each menu item. Each review has a rating, `review.stars`. For each item that appears in the review text (`review.text`), append the review's rating to a list of ratings for that item. The lists are kept in a dictionary `item_ratings`.

To get the matched phrases, you can reference the `PhraseMatcher` documentation for the structure of each match object:

>A list of `(match_id, start, end)` tuples, describing the matches. A match tuple describes a span `doc[start:end]`. The `match_id` is the ID of the added match pattern.

In [8]:
from collections import defaultdict

# item_ratings is a dictionary of lists. If a key doesn't exist in item_ratings,
# the key is added with an empty list as the value.
item_ratings = defaultdict(list)
      
for idx, review in data.iterrows():
    doc = nlp(review.text)
    matches = matcher(doc)

    found_items = set([doc[match[1]:match[2]] for match in matches])

    for item in found_items:
        item_ratings[str(item).lower()].append(review.stars)

q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# Lines below will give you a hint or solution code
#q_3.hint()
# q_3.solution()

# Step 4: What's the worst reviewed item?


Using these item ratings, find the menu item with the worst average rating.

In [13]:
# Calculate the mean ratings for each menu item as a dictionary
mean_ratings = {item: sum(ratings)/len(ratings) for item, ratings in item_ratings.items()}
print(mean_ratings)
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]
print(worst_item)
q_4.check()

{'chicken parmigiana': 4.444444444444445, 'eggplant': 3.968421052631579, 'pizza': 4.304469273743017, 'steak and cheese': 4.888888888888889, 'meatball': 4.079754601226994, 'cannoli': 4.337078651685394, 'pasta': 4.392156862745098, 'prosciutto': 4.619047619047619, 'purista': 4.641791044776119, 'cheese steak': 4.454545454545454, 'cheesesteak': 4.335616438356165, 'calzone': 4.263636363636364, 'chicken spinach salad': 4.5, 'tiramisu': 4.2592592592592595, 'italian combo': 3.909090909090909, 'italian beef': 4.0, 'salami': 4.21875, 'chicken parm': 4.155172413793103, 'turkey sandwich': 3.8, 'chicken cutlet': 3.5454545454545454, 'tuna salad': 4.0, 'chicken pesto': 4.566666666666666, 'ziti': 4.230769230769231, 'artichoke salad': 5.0, 'lasagna': 4.409638554216867, 'fettuccini alfredo': 5.0, 'pizzas': 4.393939393939394, 'turkey breast': 5.0, 'calzones': 4.552631578947368, 'mac and cheese': 4.444444444444445, 'grilled veggie': 4.5, 'garlic bread': 4.021739130434782, 'spaghetti': 3.8536585365853657, '

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
# Lines below will give you a hint or solution code
#q_4.hint()
#q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

# There are many ways to do this. Here is one.
mean_ratings = {item: sum(ratings)/len(ratings) for item, ratings in item_ratings.items()}
worst_item = sorted(mean_ratings, key=mean_ratings.get)[0]

```

In [ ]:
# After implementing the above cell, uncomment and run this to print 
# out the worst item, along with its average rating. 

#print(worst_item)
#print(mean_ratings[worst_item])

# Step 5: Are counts important here?

Similar to the mean ratings, you can calculate the number of reviews for each item.

In [ ]:
counts = {item: len(ratings) for item, ratings in item_ratings.items()}

item_counts = sorted(counts, key=counts.get, reverse=True)
for item in item_counts:
    print(f"{item:>25}{counts[item]:>5}")

Here is code to print the 10 best and 10 worst rated items. Look at the results, and decide whether you think it's important to consider the number of reviews when interpreting scores of which items are best and worst.

In [ ]:
sorted_ratings = sorted(mean_ratings, key=mean_ratings.get)

print("Worst rated menu items:")
for item in sorted_ratings[:10]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")
    
print("\n\nBest rated menu items:")
for item in sorted_ratings[-10:]:
    print(f"{item:20} Ave rating: {mean_ratings[item]:.2f} \tcount: {counts[item]}")

Run the following line after you've decided your answer.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_5.solution()

# Keep Going

Now that you are ready to combine your NLP skills with your ML skills, **[see how it's done](https://www.kaggle.com/matleonard/text-classification)**.